In [ ]:
import pandas as pd
import openpyxl as opx
import numpy as np
from datetime import date, datetime, time, timedelta
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
risp = False
while (risp == False):
    ans = input('Quale file si vuole utilizzare?\n\n1 = File Originale\n\n2 = File modificato con porzioni di viaggio tutte nella stessa data\n\nDigitare il numero dell\'opzione   ')
    if (ans == '1' or ans == '2'):
        risp = True

In [ ]:
if ans == '1':
    file = 'file di dati.xlsx'
else:
    file = 'file di dati singolo giorno.xlsx'

xlsx   = opx.load_workbook(filename = file, data_only=True)
tratte = xlsx['porzioni di viaggio']
tempi  = xlsx['tempi di percorrenza a vuoto']

df_tempi    = pd.DataFrame(np.array([[i.value for i in j] for j in  tempi['A2':'Q18']]))  
df_porzioni = pd.DataFrame(np.array([[i.value for i in j] for j in tratte['A2':'F158']])) 

xlsx.close()

In [ ]:
max_bus = int(input("\nINSERIRE il numero di autobus a disposizione del comune \nIn seguito premere INVIO:\n\n\n"))

K = set(range(1, max_bus + 1))

In [ ]:
ser_t = df_tempi.iloc[0:17, 0]

In [ ]:
df_tempi.columns = ser_t
df_tempi.index = ser_t

In [ ]:
df_tempi.columns = df_tempi.columns.fillna('drop')
df_tempi.index = df_tempi.index.fillna('drop')
df_tempi.drop('drop', inplace = True)
df_tempi.drop('drop', axis = 1, inplace = True)

In [ ]:
ser_p = df_porzioni.iloc[0, 0:6]
df_porzioni.columns = ser_p
df_porzioni.drop( 0, inplace = True)

In [ ]:
for i in range (0, 156):
    for j in range (2, 4):
        if 'medag' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "medaglie d'oro"
        if ('stazione' in df_porzioni.iat[i, j] and df_porzioni.iat[i, j] != 'autostazione'):
            df_porzioni.iat[i, j] = "stazione"
        if 'kenn' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "kennedy"
        if 'medoro' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "corti di medoro"
        if 'pontegra' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "pontegradella itis"
        if 'smm' in df_porzioni.iat[i, j]:
            df_porzioni.iat[i, j] = "smm"
        else: continue
            

idx_list = df_tempi.index.tolist()
idx = pd.Series(idx_list)
idx[5] = "medaglie d'oro"
idx = idx.replace({'stazione fs':'stazione', 'piazzale dante':'p. dante', 'santa maria maddalena': 'smm'})
df_tempi.index = idx
df_tempi.columns = idx


In [ ]:
c = np.zeros((156, 156))
I = set(range(1, 157))
for i in I:
    for j in I:
        if df_porzioni.at[i, "final stop"] == df_porzioni.at[j, "initial stop"]:
            continue
        elif df_tempi.at[df_porzioni.at[i, "final stop"], df_porzioni.at[j, "initial stop"]] is None:
                delta = df_tempi.at[df_porzioni.at[j, "initial stop"], df_porzioni.at[i, "final stop"]]
        else:
                delta = df_tempi.at[df_porzioni.at[i, "final stop"], df_porzioni.at[j, "initial stop"]]
        
        if (df_porzioni.at[i, "final time"] + timedelta(minutes = delta) <= df_porzioni.at[j, "initial time"]):
            c[i-1, j-1] = 1
        else :
            c[i-1, j-1] = 0   

C = pd.DataFrame(c)

In [ ]:
extra_pass = df_porzioni["extra passengers"]
df_extra = extra_pass.copy() 
tratte_bus = df_porzioni [['initial stop', 'final stop']]
df_tratte = tratte_bus.copy()

In [ ]:
G = nx.DiGraph()

for i in I:
    u_i = '{}. {} -> {} nodo1'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    v_i = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    G.add_edge(u_of_edge = u_i, v_of_edge = v_i, weight = 0, capacity = 1)

In [ ]:
for i in I:
    for j in I:
        if c[i-1, j-1] == 1:
            u_comp = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
            v_comp = '{}. {} -> {} nodo1'.format(j, df_tratte.at[j, 'initial stop'],df_tratte.at[j, 'final stop'])
            G.add_edge(u_of_edge = u_comp, v_of_edge = v_comp, weight = -df_extra[j] , capacity = 1)
            


In [ ]:
G.add_node('s')
G.add_node('t')
for i in I:
    u_i = '{}. {} -> {} nodo1'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    v_i = '{}. {} -> {} nodo2'.format(i, df_tratte.at[i, 'initial stop'],df_tratte.at[i, 'final stop'])
    G.add_edge(u_of_edge = 's', v_of_edge = u_i, weight = -df_extra[i], capacity = 1)
    G.add_edge(u_of_edge = v_i, v_of_edge = 't', weight = 0, capacity = 1)

In [ ]:
flow_costs_k = []

k = 0
incremento = True
while (incremento and k < max_bus):
    G.add_edge(u_of_edge = 't', v_of_edge = 's', weight = 0, capacity = k)
    flowCost, flowDict = nx.algorithms.flow.network_simplex(G, capacity ='capacity', weight ='weight')
    flow_costs_k.append((-1)*flowCost)
    
    if (flow_costs_k[k] == flow_costs_k[k-1] and k > 1):
        flow_max = flow_costs_k[k-1]
        incremento = False
    k = k + 1
    
if(incremento == False):
    for i in range(3):
        flow_costs_k.append(flow_max)

flow_edges = []

for u, v in G.edges():
    G[u][v]['weight'] = -G[u][v]['weight']
    if flowDict[u][v] != 0:
        flow_edges.append(('%s'%u,'%s'%v))


In [ ]:
ser_k = pd.Series(flow_costs_k)

ser_k = pd.DataFrame(ser_k)
ser_k = ser_k.transpose()
plot = sbn.catplot(data =ser_k, kind ="bar", aspect = 2, height =10, palette ="crest")

plot.savefig('Istogramma.png')
